In [22]:
from pymongo import MongoClient
client = MongoClient()

In [23]:
client = MongoClient('localhost', 27017)

In [24]:
mydb = client["amazonPhones"] 
my_collection = mydb['phoneReviews'] 

In [32]:
import bson
import json
import dateutil 
import parser
from bson.json_util import loads

In [35]:
with open('outputWithNPCountAndSentiment.json', 'r') as f:
    data = json.load(f)

# jsonDataSetString = open("outputWithNPCountAndSentiment.json", "r").read()
# record = json.loads(str(item)) for item in jsonDataSetString.strip().split('\)

all_reviews = []

for record in data:
    for review in record['reviewList']:
        all_reviews.append(review)

# test print
# for i in range(10):
#     print(all_reviews[i])

In [34]:
# This will throw an error (duplicate key) but it works
converted = loads(json.dumps(all_reviews))
my_collection.insert_many(converted)

BulkWriteError: batch op errors occurred, full error: {'writeErrors': [{'index': 57348, 'code': 11000, 'errmsg': "E11000 duplicate key error collection: amazonPhones.phoneReviews index: _id_ dup key: { _id: ObjectId('6416645e9a51903fe2bf93e3') }", 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6416645e9a51903fe2bf93e3')}, 'op': {'_id': ObjectId('6416645e9a51903fe2bf93e3'), 'overall': 1, 'verified': False, 'reviewTime': '12 21, 2005', 'reviewerID': 'AJMBX04BZWPSD', 'asin': 'B0000SX3BK', 'reviewerName': 'K. Horn', 'reviewText': 'I had been very happy with my Motorola phone until the battery started losing its charge within a day, even when the phone was turned off. The friendly but incompetent customer support kept me on the phone for an average of 40 minutes for the repeated calls trying to figure out the problem. After the phone had finally been sent to their technical service center, they returned it to me that the phone was irreperable "due to customer abuse." I have done no